### Execution environment
Since Jupyter notebookes work from a different directory the code block below changes the execution path to the root of the prject

In [148]:
import client as cl
import os
dir = os.getcwd()
if dir.split("/")[-3] == "codebase":
    os.chdir("../../")

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tenseal as ts
import time


In [2]:
# importing module
import logging

# Create and configure logger
logging.basicConfig(
    filename="encrypted_evaluation.log", format="%(asctime)s %(message)s", filemode="a"
)


In [ ]:
torch.random.manual_seed(61)

In [3]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out


In [4]:
def test_train_split(df):
    training_data = df.sample(frac=0.8, random_state=25)  #
    testing_data = df.drop(training_data.index)
    y_train, x_train = (
        training_data["YearsExperience"].to_numpy(),
        training_data["Salary"].to_numpy(),
    )
    y_test, x_test = (
        testing_data["YearsExperience"].to_numpy(),
        testing_data["Salary"].to_numpy(),
    )
    return x_train, y_train, x_test, y_test


In [5]:
df = pd.read_csv("./LinearRegression/Data/Custom_Salary_Data.csv")
sns.scatterplot(data=df, x="Salary", y="YearsExperience")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './LinearRegression/Data/Custom_Salary_Data.csv'

In [ ]:
x_train, y_train, x_test, y_test = test_train_split(df)
x_train = x_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
x_test = x_test.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)


In [ ]:
model = LinearRegressionModel(1, 1)  # single dimension
# criterion = nn.MSELoss(reduction = "mean") # mean squared error, minimise total loss
criterion = nn.L1Loss(reduction="mean")  # mean absolute error, minimise total loss
learning_rate = 5e-10
optimizer = torch.optim.SGD(
    model.parameters(), lr=learning_rate
)  # Stochastic Gradient Descent
EPOCHS = 100000
model = model.double()


In [ ]:
# Convert numpy array to torch Variable
x_train_tensor = torch.from_numpy(x_train).double().requires_grad_()
y_train_tensor = torch.from_numpy(y_train).double().requires_grad_()

x_test_tensor = torch.from_numpy(x_test).double().requires_grad_()
y_test_tensor = torch.from_numpy(y_test).double().requires_grad_()


In [ ]:
def train(model, optimizer, criterion, inputs, labels, epochs=EPOCHS):
    for epoch in range(epochs + 1):
        # Forward to get output
        outputs = model(inputs)
        # Calculate Loss
        loss = criterion(outputs, labels)
        # Getting gradients w.r.t. parameters
        loss.backward()
        # Updating parameters
        optimizer.step()
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        if epoch % 2000 == 0:
            print("epoch {}, loss {}".format(epoch, loss.item()))
        if loss.item() < 0.01:
            break
    global Final_LOSS
    Final_LOSS = loss.item()
    return model


In [ ]:
model = train(model, optimizer, criterion, x_train_tensor, y_train_tensor, EPOCHS)

epoch 0, loss 10530.06305071712
epoch 2000, loss 2235.9030201410724
epoch 4000, loss 2.6359817477351646
epoch 6000, loss 2.642207844289872
epoch 8000, loss 2.626195183893078
epoch 10000, loss 2.632421280447787
epoch 12000, loss 2.6386473770024947
epoch 14000, loss 2.622634716605702
epoch 16000, loss 2.6288608131604088
epoch 18000, loss 2.6350869097151173
epoch 20000, loss 2.6413130062698245
epoch 22000, loss 2.6253003458730317
epoch 24000, loss 2.63152644242774
epoch 26000, loss 2.6377525389824474
epoch 28000, loss 2.621739878585655
epoch 30000, loss 2.627965975140362
epoch 32000, loss 2.6341920716950704
epoch 34000, loss 2.640418168249778
epoch 36000, loss 2.6244055078529844
epoch 38000, loss 2.6306316044076925
epoch 40000, loss 2.636857700962401
epoch 42000, loss 2.6430837975171073
epoch 44000, loss 2.627071137120315
epoch 46000, loss 2.6332972336750236
epoch 48000, loss 2.6395233302297303
epoch 50000, loss 2.6235106698329385


In [ ]:
def round_tensor_to_multiple(tensor, base):
    numpy_arr = tensor.numpy()
    rounded_np_arr = np.round(base * np.round(numpy_arr / base))
    return torch.from_numpy(rounded_np_arr)

In [ ]:
def accuracy(model, x, y):
    t_start = time.time()
    with torch.no_grad():  # no need to calculate gradients for testing
        y_pred_experience = model(x)
        y_pred_experience = torch.round(
            y_pred_experience, decimals=-1
        )  # nearest year experience rounded to whole number
        y_real_rounded = torch.round(
            y, decimals=-1
        )  # real salary (rounded unit: the thousands) LHS of the decimal
        accuracy = torch.eq(y_pred_experience, y_real_rounded)
    t_end = time.time()
    global duration
    duration = t_end - t_start
    return torch.sum(accuracy).item() / len(accuracy) * 100


In [ ]:
plain_accuracy = accuracy(model, x_test_tensor, y_test_tensor)
print(f"Accuracy on plain test_set: {plain_accuracy}, duration: {duration}")

Accuracy on plain test_set: 86.0, duration: 0.0005915164947509766


In [ ]:
class EncryptedLR:
    def __init__(self, torch_linear_model):
        # TenSEAL processes lists and not torch tensors,
        # so we take out the parameters from the PyTorch model
        self.weight = torch_linear_model.linear.weight.data.tolist()[0]
        self.bias = torch_linear_model.linear.bias.data.tolist()
        

    def forward(self, enc_x):
        #Propogate the network
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    ################################################
    ## You can use the functions below to perform ##
    ## the evaluation with an encrypted model     ##
    ################################################

    def encrypt(self, key_context):
        self.weight = ts.ckks_vector(key_context, self.weight)
        self.bias = ts.ckks_vector(key_context, self.bias)

    def decrypt(self, key_context):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()


In [ ]:
def encrypted_evaluation(model, enc_x_test, y_test):
    t_start = time.time()
    with torch.no_grad():
        y_pred_enc = model(enc_x_test)
        y_pred_output = y_pred_enc.decrypt()
        #Converting PlainTensor into pytorch tensor https://github.com/OpenMined/TenSEAL/blob/main/tutorials/Tutorial%202%20-%20Working%20with%20Approximate%20Numbers.ipynb
        y_pred_output = y_pred_output.tolist()
        y_pred_output = torch.FloatTensor(y_pred_output)
        ##################################################
        y_pred_output = torch.round(y_pred_output, decimals=-1)
        y_real_rounded = torch.round(y_test, decimals=-1) # Rounding the Years of experience to the nearest whole number
        y_real_rounded = torch.flatten(y_real_rounded)
        accuracy = torch.eq(y_pred_output, y_real_rounded)
    t_end = time.time()
    global encrypted_duration
    encrypted_duration = t_end - t_start
    correct_guess = torch.sum(accuracy).item()
    return correct_guess / len(accuracy) * 100


In [ ]:
encrypted_linear_regression_model = EncryptedLR(model)
encrypted_x_test_tensor = ts.ckks_tensor(cl.key_context, x_test_tensor.detach().numpy())

In [ ]:
E_accuracy = encrypted_evaluation(
    encrypted_linear_regression_model, encrypted_x_test_tensor, y_test_tensor
)
print(f"Accuracy on encrypted test_set: {E_accuracy}, duration: {encrypted_duration}")


Accuracy on encrypted test_set: 86.0, duration: 0.14290189743041992


In [ ]:
diff_accuracy = plain_accuracy - E_accuracy
print(f"Difference between plain and encrypted accuracies: {diff_accuracy}")
if diff_accuracy < 0:
    print(
        "Oh! We got a better accuracy on the encrypted test-set! The noise was on our side..."
    )

Difference between plain and encrypted accuracies: 0.0


In [ ]:
logger = logging.getLogger()

# Setting the threshold of logger to DEBUG
logger.setLevel(logging.DEBUG)
logger.debug(
    f"Epoch number:{EPOCHS} | Learning rate: {learning_rate} | Final Loss: {Final_LOSS} | Plain test_set accuracy: {plain_accuracy} | Encrypted test_set accuracy: {E_accuracy if E_accuracy != 0 else 'N/A'}"
)
